In [2]:
from cartoframes.auth import set_default_credentials
from cartoframes import read_carto, to_carto
import geopandas as gpd
import pandas as pd
import os
import logging 
from shapely import geometry
import requests
import re
from bs4 import BeautifulSoup
import glob
from zipfile import ZipFile
import shutil


NumExpr defaulting to 8 threads.


In [ ]:
# set up logging

# get top-level logger object
logger = logging.getLogger()
for handler in logger.handlers: logger.removeHandler(handler)
# manually set level 
logger.setLevel(logging.INFO)
# print to console
console = logging.StreamHandler()
logger.addHandler(console)
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [4]:
data_dir = "data"

In [5]:
logger.info('Pull MPAs from OneDrive')
# read the mpa data set
raw_data_file = glob.glob(os.path.join(os.getenv("OCEANWATCH_DATA_DIR"),'WDPA_WDOECM_*_Public_marine_shp.zip'))[0]
dest_dir = os.path.join(data_dir, os.path.basename(raw_data_file))[:-4]
raw_data_file_unzipped = os.path.basename(dest_dir)
shutil.copy(raw_data_file, dest_dir)l

Pull MPAs from OneDrive


'data/WDPA_WDOECM_Aug2021_Public_marine_shp'

In [7]:
zip_ref = ZipFile(raw_data_file, 'r')
zip_ref.extractall(data_dir)
zip_ref.close()

# find all the zipped folders that contain the shapefiles
zipped_shp = glob.glob(os.path.join(data_dir, '*shp_*.zip' ))
# unzipped each of them
for zipped in zipped_shp:
    zip_ref = ZipFile(zipped, 'r')
    zip_ref.extractall(os.path.join(data_dir, zipped.split('.')[0][-5:]))
    zip_ref.close()

In [8]:
# store the path to all the polygon shapefiles in a list
mpa_files = [glob.glob(os.path.join(data_dir, zipped.split('.')[0][-5:], '*polygons.shp'))[0] for zipped in zipped_shp]
gdf_list=[]

In [9]:
for file in mpa_files:
    gdf = gpd.read_file(file)
    print('Length MPA = ' + str(len(gdf.index)))
    gdf_list.append(gdf) 

Length MPA = 5832


Error in fread() reading object of size 2970728 at offset 220709472 from .shp file
Error in fread() reading object of size 913812 at offset 223680200 from .shp file
Error in fread() reading object of size 109364 at offset 224594012 from .shp file
Error in fread() reading object of size 1249104 at offset 224703376 from .shp file
Error in fread() reading object of size 734032 at offset 225952480 from .shp file
Error in fread() reading object of size 188384 at offset 226686512 from .shp file
Error in fread() reading object of size 1255568 at offset 226874896 from .shp file
Error in fread() reading object of size 7277884 at offset 228130464 from .shp file
Error in fread() reading object of size 1767152 at offset 235408348 from .shp file
Error in fread() reading object of size 101308 at offset 237175500 from .shp file
Error in fread() reading object of size 313200 at offset 237276808 from .shp file
Error in fread() reading object of size 131208 at offset 237590008 from .shp file
Error in fr

In [12]:
logger.info('Merge MPAs')
# merge geopandas dataframes 
gdf_mpa = gpd.GeoDataFrame(pd.concat(gdf_list))
print('Length total MPA = ' + str(len(gdf_mpa.index)))

Merge MPAs


Length total MPA = 17494


In [13]:
# filter out MPAs that are Proposed or who's status is not designated
gdf_mpa = gdf_mpa.loc[gdf_mpa['STATUS'] != 'Proposed']
print('Length subset MPA = ' + str(len(gdf_mpa.index)))

Length subset MPA = 17281
   WDPAID WDPA_PID PA_DEF                                  NAME  \
0     1.0        1      1  Diamond Reef and Salt Fish Tail Reef   
1     2.0        2      1                         Palaster Reef   
2    27.0       27      1                             Folkstone   
3    46.0       46      1      Reserva Biológica Atol Das Rocas   
4    57.0       57      1        Parque Nacional Do Cabo Orange   

                          ORIG_NAME              DESIG           DESIG_ENG  \
0                      Diamond Reef     Marine Reserve      Marine Reserve   
1                     Palaster Reef     Marine Reserve      Marine Reserve   
2                         Folkstone     Marine Reserve      Marine Reserve   
3  Reserva Biológica Atol Das Rocas  Reserva Biológica  Biological Reserve   
4    Parque Nacional Do Cabo Orange             Parque                Park   

  DESIG_TYPE IUCN_CAT        INT_CRIT  ...  \
0   National       Ia  Not Applicable  ...   
1   Nation

In [1]:
gdf_mpa['valid']= gdf_mpa.is_valid 
gdf_mpa = gdf_mpa.loc[gdf_mpa['valid'] == True]

NameError: name 'gdf_mpa' is not defined

In [ ]:
print('Length valid subset MPA = ' + str(len(gdf_mpa.index)))

In [ ]:
# save processed dataset to shapefile
gdf_mpa.to_file('valid_mpa.shp',driver='ESRI Shapefile')